In [ ]:
import re
import pandas as pd
import os
import numpy as np


# author: 卢克治
# 完成对文本中所有满足条件的结果进行全部替换
def clean_replace_all(content,find_str,replace_str):
    while content.find(find_str) > 0:
        content = content.replace(find_str, replace_str)
    return content


# Author：卢克治
# Function：完成对content内容中的html格式重新清洗，添加上标点符号
def clean_content_addPunctuation(content):
    #lucas 整理需要直接替换为空的标识符集合
    sub_to_space_list = ["<b>","&nbsp;","<u>","<span>","<strong>","<label>","<span lang=EN-US>","\u3000","<p></p>","</span>","</font>","</FONT>","*"]
    #lucas 整理需要直接替换为句号-"。"的标识符集合
    sub_to_fullStop_list = ["</b>","</u>","</label>", "</a>","<br>","</br>","</div>","</title>"]
    #lucas 整理需要直接替换为分号-"；"的标识符集合
    sub_to_semicolon_list = []
    # lucas 整理需要直接替换为逗号-"，"的标识符集合
    sub_to_comma_list = ["</p>","</strong>"]

    #lucas 按照以上符号类别进行替换
    for sub_item in sub_to_space_list:
        content = content.replace(sub_item, "")
    for sub_item in sub_to_fullStop_list:
        content = content.replace(sub_item, "。")
    for sub_item in sub_to_semicolon_list:
        content = content.replace(sub_item, "；")
    for sub_item in sub_to_comma_list:
        content = content.replace(sub_item, "，")

    # 去除标签<u>、空格、&nbsp;、特定标签
    content = re.sub(r'\<span.*?\>', "", content)
    content = re.sub(r'\<font.*?\>', "", content)
    content = re.sub(r'\<FONT.*?\>', "", content)
    content = re.sub(r'\<a.*?\>', "", content)


    # content = content.encode('gbk', 'ignore').decode('gbk', 'ignore').encode('utf8', 'ignore').decode('utf8',
    #                                                                                                   'ignore')  # 去除一些特殊的符号

    ss = re.sub('<[^>]+>', '#', content) # 去除<>包含的内容
    clean_content = ss.split("#")
    clean_content = [i.replace("\n", "").replace("\r", "").replace("\t", " ") for i in clean_content]  # 去除换行和tab
    clean_content = [re.sub(r'[@a-zA-Z0-9-\.：; \t]{3,}\{.*?\}', '', i) for i in clean_content]  # 去除CSS格式
    clean_content = [i.strip() for i in clean_content]
    clean_content = [i.replace(" ", "") for i in clean_content if i != ""]
    clean_content = [i for i in clean_content if i != ""]
    clean_content = [i.replace(":", "：") for i in clean_content]
    clean_content = [i.replace("？", "") for i in clean_content]


    #lucas 添加最终输出文本格式的过滤
    clean_content_result = "".join(clean_content)
    clean_content_result = clean_replace_all(clean_content_result," ", "")  # lucas 直接将空格全部删除
    clean_content_result = clean_replace_all(clean_content_result,"：，", "：")    # lucas 将：，符号删除逗号。
    clean_content_result = clean_replace_all(clean_content_result, "，：", "：")   # lucas 将，：符号删除逗号。
    clean_content_result = clean_replace_all(clean_content_result,"。。", "。")
    clean_content_result = clean_replace_all(clean_content_result,"，，", "，")
    clean_content_result = clean_replace_all(clean_content_result,"：。", "：")    # lucas 将：。符号删除句号。
    clean_content_result = clean_replace_all(clean_content_result, "。：", "：")
    clean_content_result = clean_replace_all(clean_content_result,"。，", "。")
    clean_content_result = clean_replace_all(clean_content_result,"，。", "。")
    clean_content_result = clean_replace_all(clean_content_result,",", "，")

    return [clean_content_result]


# lucas
# 清洗html格式主函数
def clean_html_format(input_file,output_file):
    result_wrong_pd = pd.read_csv(input_file,
                                  names=['0', 'id', "2", "3", "4", "5", "6", "content", "8", "9", "10", "11", "12"],
                                  usecols=["id", "content"], encoding="gb18030")
    result_clean = [clean_content(str(c)) for c in result_wrong_pd["content"]]
    result_clean_lucas = [clean_content_addPunctuation(str(c)) for c in result_wrong_pd["content"]]
    for i, line in enumerate(result_clean):
        temp = ""
        for elem in line:
            temp += elem
            temp += " "
        result_clean[i] = temp
    for i, line in enumerate(result_clean_lucas):
        temp = ""
        for elem in line:
            temp += elem
            temp += " "
        result_clean_lucas[i] = temp
    # lucas将id信息和清洗后的数据进行拼接
    result_id = result_wrong_pd["id"]
    result_filter_content = pd.DataFrame(result_clean, columns=["filter_content"])
    result_filter_content_lucas = pd.DataFrame(result_clean_lucas, columns=["filter_content_lucas"])
    filter_result = pd.concat(
        [result_id, result_wrong_pd["content"], result_filter_content, result_filter_content_lucas], axis=1)
    filter_result.to_csv(output_file, index=None)
    del filter_result #lucas 释放变量占用的内存
    return



# lucas
# 读取处理完html并加上标点符号的csv文件，将其中第四列-含标点的结果进行汇总
def huizong_html_format(input_file,output_file):
    result_wrong_pd = pd.read_csv(input_file,
                                  header=0,
                                  usecols=["id", "filter_content_lucas"], encoding="utf-8")
    result_wrong_pd.to_csv(output_file,mode="a", index=None,header=False)
    del result_wrong_pd #lucas 释放变量占用的内存
    return


# lucas
# 读取处理完html并加上标点符号的csv文件，将其中第四列-含标点的结果进行汇总
def huizong_yixuan_wrong(input_file,output_file):
    result_wrong_pd = pd.read_csv(input_file,
                                  header=0,
                                  usecols=["id", "purchaser_new","supplier_new"], encoding="utf-8")
    result_wrong_pd.to_csv(output_file,mode="a", index=None,header=False)
    del result_wrong_pd #lucas 释放变量占用的内存
    return



#@author lucas
#@function 完成清洗完html的文件读取形成一个总体的文件
def huizong_html_format_main(file_path,save_path):
    path_list = os.listdir(file_path)
    for i, folder in enumerate(path_list):
        # for i,path in enumerate(path_list[4:]):  #当程序出现错误时，修改n，从错误的地方再开始运行,n可根据记录的i值获得。
        folder_list = os.listdir(file_path + '/'+ folder)
        for i, file in enumerate(folder_list):
            if os.path.isfile(file_path + '/'+folder+'/'+ file):  # 判断是否为文件
                cur_file_path = file_path + '/'+folder+'/'+ file
                huizong_html_format(cur_file_path, save_path)

#@author lucas 20200922
#@function 完成对艺轩所有wrong文件中采购商为空的id数据进行汇总
def huizong_yixuan_wrong_main(file_path,save_path):
    path_list = os.listdir(file_path)
    for i, folder in enumerate(path_list):
        # for i,path in enumerate(path_list[4:]):  #当程序出现错误时，修改n，从错误的地方再开始运行,n可根据记录的i值获得。
        folder_list = os.listdir(file_path + '/'+ folder)
        for i, file in enumerate(folder_list):
            if os.path.isfile(file_path + '/'+folder+'/'+ file):  # 判断是否为文件
                cur_file_path = file_path + '/'+folder+'/'+ file
                huizong_yixuan_wrong(cur_file_path, save_path)



#@author lucas 20200922
#@function 完成对艺轩所有wrong文件中采购商为空的id读取后，到处理完的所有id和加上标点的content中对应content进行输出汇总需要进行
def filter_yixuan_caigoushang_wrong_content_main(yixuan_wrong_file, all_content_file, save_path):
    #lucas 以第一列为id索引读取所有含有标点的content内容
    all_content_dic = pd.read_csv(all_content_file, index_col=0, names=["id","filter_content_lucas"])
    all_content_dic = all_content_dic.drop_duplicates(keep="first")
    all_id = all_content_dic.index

    #lucas 读取艺轩那边汇总完的所有的wrong数据id
    all_yixuan_wrong_dic = pd.read_csv(yixuan_wrong_file, index_col=0, names=["id", "purchaser_new", "supplier_new"])
    save_yixuan_wrong_content = pd.DataFrame(columns=["id","filter_content_lucas"])
    for index, row in all_yixuan_wrong_dic.iterrows():
        if row["purchaser_new"] is pd.np.nan:
            if index in all_id:
                cur_content = all_content_dic.loc[index,"filter_content_lucas"]
                # save_yixuan_wrong_content.append([{"id":index,"filter_content_lucas":cur_content}],ignore_index=True)
                save_yixuan_wrong_content.loc[index]=[index,cur_content]
    save_yixuan_wrong_content.to_csv(save_path, mode="a", index=False, header=True)



#@lucas 20200917
#@fuction 主要先遍历服务器上文件夹下面所有文件，然后去除html格式
def clean_html_format_main(file_path,save_path):
    path_list = os.listdir(file_path)
    for i, folder in enumerate(path_list):
        # for i,path in enumerate(path_list[4:]):  #当程序出现错误时，修改n，从错误的地方再开始运行,n可根据记录的i值获得。
        folder_list = os.listdir(file_path + '/'+ folder)
        for i, file in enumerate(folder_list):
            if os.path.isfile(file_path + '/'+folder+'/'+ file):  # 判断是否为文件
                cur_file_path = file_path + '/'+folder+'/'+ file
                clean_html_format(cur_file_path, save_path+ '/'+folder+'/'+ file)

#@lucas 20200917
#@fuction 将服务器上运行完的清洗html格式的结果进行汇总
def clean_html_format_huizong(file_path,save_path):
    path_list = os.listdir(file_path)
    for i, folder in enumerate(path_list):
        # for i,path in enumerate(path_list[4:]):  #当程序出现错误时，修改n，从错误的地方再开始运行,n可根据记录的i值获得。
        folder_list = os.listdir(file_path + '/'+ folder)
        for i, file in enumerate(folder_list):
            if os.path.isfile(file_path + '/'+folder+'/'+ file):  # 判断是否为文件
                cur_file_path = file_path + '/'+folder+'/'+ file
                clean_html_format(cur_file_path, save_path+ '/')

#@author 卢克治
#@function 飞跃他们原始的清洗函数
def clean_content(content):  # 预清洗中标公告，将元素放在一个数组列表里
    sub_list = ["<u>", "&nbsp;", "</u>", "<span lang=EN-US>", "</span>", "<span>", "\u3000", "<b>", "</b>", "<strong>",
                "</strong>", "</font>", "</FONT>", "<label>", "</label>", "</a>"]
    # 去除标签<u>、空格、&nbsp;、特定标签
    for sub in sub_list:
        content = content.replace(sub, "")
    content = re.sub(r'\<span.*?\>', "", content)
    content = re.sub(r'\<font.*?\>', "", content)
    content = re.sub(r'\<FONT.*?\>', "", content)
    content = re.sub(r'\<a.*?\>', "", content)

    content = content.encode('gbk', 'ignore').decode('gbk', 'ignore').encode('utf8', 'ignore').decode('utf8',
                                                                                                      'ignore')  # 去除一些特殊的符号
    ss = re.sub('<[^>]+>', '#', content)
    clean_content = ss.split("#")  # 去除<>包含的内容
    clean_content = [i.replace("\n", "").replace("\r", "").replace("\t", " ") for i in clean_content]  # 去除换行和tab
    clean_content = [re.sub(r'[@a-zA-Z0-9-\.：; \t]{3,}\{.*?\}', '', i) for i in clean_content]  # 去除CSS格式
    clean_content = [i.strip() for i in clean_content]
    clean_content = [i.replace(" ", "") for i in clean_content if i != ""]
    clean_content = [i for i in clean_content if i != ""]
    clean_content = [i.replace(":", "：") for i in clean_content]
    clean_content = [i.replace("？", "") for i in clean_content]
    return clean_content


if __name__ == '__main__':
    # func1
    #----------------------完成服务器172.20.47.51下extractbids中文件的html格式清洗-------------------------
    # lucas 先读取文件,遍历双层文件夹
    # file_path = "/data/nfs/extractbids"  # 原始中标数据文件夹路径
    # save_path = "outputData/html_clean_lucas"  # 存放提取出中标信息的文件夹(/data/nfs/lucas/数据清洗/outputData/html_clean_lucas)
    # clean_html_format_main(file_path, save_path)    # lucas 0917 调用html去除格式主函数

    # func2
    # ----------------------完成服务器172.20.47.51下/data/nfs/lucas/数据清洗/outputData/html_clean_lucas清洗了html结果进行结果汇总到一个csv文件-------------------------
    # file_path = "/data/nfs/lucas/数据清洗/outputData/html_clean_lucas"  # 清洗完的数据文件夹
    # save_path = "/data/nfs/lucas/数据清洗/outputData/html_clean_lucas结果汇总.csv"  # 汇总的文件夹路径
    # # lucas先输出标题，再进行内容的输出
    # title = pd.DataFrame(["id","filter_content_lucas"]).T
    # title.to_csv(save_path, mode="a", index=None, header=False)
    # huizong_html_format_main(file_path, save_path)

    #func3
    # ----------------------完成服务器10.10.5.121下/data/yixuan/identify_clean_data_wrong进行结果汇总到一个csv文件-------------------------
    # file_path = "/data/yixuan/identify_clean_data_wrong"  # 清洗完的数据文件夹
    # save_path = "/data/lucas/汇总艺轩的wrong数据/yixuan_wrong.csv"  # 汇总的文件夹路径
    # # lucas先输出标题，再进行内容的输出
    # title = pd.DataFrame(["id", "purchaser_new","supplier_new"]).T
    # title.to_csv(save_path, mode="a", index=None, header=False)
    # huizong_yixuan_wrong_main(file_path, save_path)

    # func4
    # ----------------------完成服务器10.10.5.121下/data/lucas/汇总艺轩的wrong数据/yixuan_wrong结果汇总.csv中提取艺轩处理的wrong数据-------------------------
    input_wrong_file = "/data/lucas/汇总艺轩的wrong数据/yixuan_wrong结果汇总.csv"
    input_content_file = "/data/lucas/汇总艺轩的wrong数据/input/html_clean_lucas结果汇总.csv"
    output_file = "/data/lucas/汇总艺轩的wrong数据/output/yixuan_wrong采购方为空原文汇总.csv"
    # lucas先输出标题，再进行内容的输出
    # title = pd.DataFrame(["id", "filter_content_lucas"]).T
    # title.to_csv(output_file, mode="a", index=None, header=False)
    filter_yixuan_caigoushang_wrong_content_main(input_wrong_file,input_content_file, output_file)
